# Marketing Analytics: Customer Segmentation 
<a id = "table-of-contents"></a>
# Table of Contents
- [Introduction](#intro)
- [Data](#data)
- [Data Preparation and Cleaning](#preparation)
- [Data Exploration](#exploration)
- [Feature Engineering](#feature_eng)
- [K-Means Clustering](#kmeans)
- [Exploration of Clusters](#exp_clust)
- [Results](#results)
- [Recommendations](#recommendations)

<a id = "intro"></a>
# Introduction
Customer segmentation is a powerful marketing technique that involves dividing a customer base into distinct segments based on shared characteristics, behaviours, or demographics. The primary purpose of customer segmentation is to better understand and serve customers in a more personalized and targeted way. Marketing segmentation helps to understand customer needs better  and reach the right customer with right messaging.  
Exploratory Data Analysis (EDA) is a necessary preliminary step before using a segmentation algorithm.

<a id = "data"></a>
# Data
The data contains 2,205 observations and 39 columns. The dataset description on the card does not match the actual columns in the dataset. The below list contains actual columns from the dataset and the assumed descriptions from the column's names. 

| Feature | Description | Comment |
|---------|-------------|---------|
| **AcceptedCmp1** | 1 if customer accepted the offer in the 1st campaign, 0 otherwise | |
| **AcceptedCmp2** | 1 if customer accepted the offer in the 2nd campaign, 0 otherwise | |
| **AcceptedCmp3** | 1 if customer accepted the offer in the 3rd campaign, 0 otherwise | |
| **AcceptedCmp4** | 1 if customer accepted the offer in the 4th campaign, 0 otherwise  | |
| **AcceptedCmp5** | 1 if customer accepted the offer in the 5th campaign, 0 otherwise  | |
| **AcceptedCmpOverall** | overall number of accepted campaigns |This column was added from the list of actual columns|
| **Response** | 1 if customer accepted the offer in the last campaign, 0 otherwise  | |
| **Complain** | 1 if customer complained in the last 2 years  | |
| **DtCustomer** | date of customer’s enrolment with the company |There is no such column in the dataset|
| **Customer_Days** | number of days since registration as a customer | |
| **Education** | customer’s level of education |There is no such column in the actual dataset|
| **education_2n Cycle** | customer has secondary education | This column was added from the list of actual columns|
| **education_Basic** | customer has basic education | This column was added from the list of actual columns|
| **education_Graduation** |Customer has a bachelor degree | This column was added from the list of actual columns|
| **education_Master** | Customer has a masters degree|This column was added from the list of actual columns|
| **education_PhD** | Customer has a PhD | This column was added from the list of actual columns |
| **Marital** | customer’s marital status. |There is no such column in the actual dataset |
| **marital_Divorced** | 1 if customer is divorced, 0 otherwise. | This column was added from the list of actual columns|
| **marital_Married** | 1 if customer is married, 0 otherwise.| This column was added from the list of actual columns|
| **marital_Single** | 1 if customer is single, 0 otherwise. |This column was added from the list of actual columns|
| **marital_Together** | 1 if customer is in relationship, 0 otherwise.| This column was added from the list of actual columns|
| **marital_Widow** | 1 if customer is a widow / widower, 0 otherwise  | |
| **Kidhome** | number of small children in customer’s household | |
| **Teenhome** | number of teenagers in customer’s household | |
| **Income** | customer’s yearly household income | |
| **MntFishProducts** | amount spent on fish products in the last 2 years | |
| **MntMeatProducts** | amount spent on meat products in the last 2 years | |
| **MntFruits** | amount spent on fruits products in the last 2 years | |
| **MntSweetProducts** | amount spent on sweet products in the last 2 years | |
| **MntWines** | amount spent on wine products in the last 2 years | |
| **MntGoldProds** | amount spent on gold products in the last 2 years | |
| **NumDealsPurchases** | number of purchases made with discount | |
| **NumCatalogPurchases** | number of purchases made using catalogue | |
| **NumStorePurchases** | number of purchases made directly in stores | |
| **NumWebPurchases** | number of purchases made through company’s web site | |
| **NumWebVisitsMonth** | number of visits to company’s web site in the last month | |
| **Recency** | number of days since the last purchase | |
| **Z_CostContact** | | This column was added from the list of actual columns|
| **Z_Revenue** | | This column was added from the list of actual columns|
| **Age** | Age of customer|  This column was added from the list of actual columns|
| **MntTotal** | Total amount spent on all the products| This column was added from the list of actual columns |
| **MntRegularProds** | | This column was added from the list of actual columns|


<a id = "preparation"></a>
# Data Preparation and Cleaning
In this section:
- Reviewing data columns and comparing them to the dataset description
- Looking for missing values
- Checking column types
- Assessing unique values


In [ ]:
#Importing necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pointbiserialr

In [ ]:
#Reading the data
data = pd.read_csv('/kaggle/input/marketing-data/ifood_df.csv')

#Taking a look at the top 5 rows of the data
data.head()

There is some data in the dataframe. Let's dig into the data.

## Reviewing data columns and comparing them to the dataset description


Retrieving the list of actual columns to compare with the column's description in the dictionary.png. The list of columns has been updated and the data description contains actual columns.

In [ ]:
data.columns

## Looking for missing values
Surprisingly, there is no missing values in the data and there are 2,205 observations in the data frame.

In [ ]:
data.isna().sum()

## Checking column types
All column types look good. There is no need to change any data types.

In [ ]:
data.info()

## Assessing unique values
Let's check the unique values in each column. If a column has the same values then we cannot use this column in our analysis and can remove it from the data frame. 

In [ ]:
data.nunique()

Columns Z_CostContact and Z_Revenue have all the same values. These columns will not help us to understand our customers better. We can drop these columns from the data frame.

In [ ]:
data.drop(columns=['Z_CostContact','Z_Revenue'],inplace=True)

<a id="exploration"></a>    
# Data Exploration
In this section:
- Box plot for the total amount spent on all products (MntTotal)
- Outliers
- Box plot and histogram for income
- Histogram for age
- Correlation matrix
- Point-Biserial correlations for binary variables

## Box plot for the total amount spent on all products (MntTotal)
Our analysis will be focused on total amount spent on all products (MntTotal). Boxplot will help us to find outliers if any.

In [ ]:
plt.figure(figsize=(6, 4))  
sns.boxplot(data=data, y='MntTotal')
plt.title('Box Plot for MntTotal')
plt.ylabel('MntTotal')
plt.show()

## Outliers
The box plot spotted a few outliers in the MntTotal. Let's take a closer look at the outliers. 

In [ ]:
Q1 = data['MntTotal'].quantile(0.25)
Q3 = data['MntTotal'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = data[(data['MntTotal'] < lower_bound) | (data['MntTotal'] > upper_bound)]
outliers.head()

### Outliers removal

In [ ]:
data = data[(data['MntTotal'] > lower_bound) & (data['MntTotal'] < upper_bound)]
data.describe()

## Box plot and histogram for income

In [ ]:
plt.figure(figsize=(6, 4))  
sns.boxplot(data=data, y='Income', palette='viridis')
plt.title('Box Plot for Income')
plt.ylabel('Income')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))  
sns.histplot(data=data, x='Income', bins=30, kde=True)
plt.title('Histogram for Income')
plt.xlabel('Income')
plt.ylabel('Frequency')
plt.show()

Income distribution is close to normal distribution with no outliers.

## Histogram for age

In [ ]:
plt.figure(figsize=(8, 6))  
sns.histplot(data=data, x='Age', bins=30, kde=True)
plt.title('Histogram for Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print("Skewness: %f" % data['Age'].skew())
print("Kurtosis: %f" % data['Age'].kurt())

The age distribution looks approximately symmetrical and the left and right sides of distribution are roughly equal. Skewness of 0.09 (close to zero) supports the visual observation of the distribution.
Kurtosis of -0.8 suggests that the distribution is close to normal with lighter tails and less peaked than a normal distribution. 

## Correlation matrix

There are many columns in the data. The correlation matrix will be very crowded if we use all columns of the data frame. We will group the columns and explore correlation between columns in each group and the column 'MntTotal'. We will focus on the column 'MntTotal' to understand how we can segment the customers who buy the most in overall. We can run similar analysis for every type of product.

In [ ]:
cols_demographics = ['Income','Age']
cols_children = ['Kidhome', 'Teenhome']
cols_marital = ['marital_Divorced', 'marital_Married','marital_Single', 'marital_Together', 'marital_Widow']
cols_mnt = ['MntTotal', 'MntRegularProds','MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
cols_communication = ['Complain', 'Response', 'Customer_Days']
cols_campaigns = ['AcceptedCmpOverall', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
cols_source_of_purchase = ['NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']
cols_education = ['education_2n Cycle', 'education_Basic', 'education_Graduation', 'education_Master', 'education_PhD']

In [ ]:
corr_matrix = data[['MntTotal']+cols_demographics+cols_children].corr()
plt.figure(figsize=(6,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

MntTotal has strong positive correlation with income and intermediate negative correlation with Kidhome. Income feature has nearly the same negative correlation with Kidhome and MntTotal.

## Point-Biserial correlations for binary variables
Pearson correlation measures the strength and direction of a linear relationship between two continuous variables. We used Pearson correlation for MntTotal, Age and Income. When we try to understand the relationship between a continuous variable MntTotal and binary variables like marital status then we should use Point-Biserial Correlation
Point-Biserial Correlation is used to measure the strength and direction of the linear relationship between a binary variable and a continuous variable.

In [ ]:
for col in cols_marital:
    correlation, p_value = pointbiserialr(data[col], data['MntTotal'])
    print(f'{correlation:.4f}: Point-Biserial Correlation for {col} with p-value {p_value:.4f}')    

There is no strong Point-Biserial correlation between MntTotal and different marital statuses. Some feature engineering may be required during the modelling process.

In [ ]:
for col in cols_education:
    correlation, p_value = pointbiserialr(data[col], data['MntTotal'])
    print(f'{correlation:.4f}: Point-Biserial Correlation for {col} with p-value {p_value:.4f}')    

There is no strong Point-Biserial correlation between MntTotal and various education levels.

<a id="feature_eng"></a>
# Feature Engineering
In this section:
- New feature: Marital
- New feature: In_relationship


## New feature: Marital
The data frame contains 5 columns to reflect marital status. We are going to create a new column 'marital' with values: Divorced, Married, Single, Together, Widow. This column will allow us to draw some additional plots. 

In [ ]:
def get_marital_status(row):
    if row['marital_Divorced'] == 1:
        return 'Divorced'
    elif row['marital_Married'] == 1:
        return 'Married'
    elif row['marital_Single'] == 1:
        return 'Single'
    elif row['marital_Together'] == 1:
        return 'Together'
    elif row['marital_Widow'] == 1:
        return 'Widow'
    else:
        return 'Unknown'
data['Marital'] = data.apply(get_marital_status, axis=1)


In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='Marital', y='MntTotal', data=data, palette='viridis')
plt.title('MntTotal by marital status')
plt.xlabel('Marital status')
plt.ylabel('MntTotal')

## New feature: In_relationship
There are 3 features that reflect if a person is single (Single, Divorced, Widow) and 2 features if a person is in relationship (Together, Married). We will add an additional feature 'In_relationship'. This feature will equal 1 if a customer's marital status is 'Married' or 'Together' and 0 in all other cases.

In [ ]:
def get_relationship(row):
    if row['marital_Married'] ==1:
        return 1
    elif row['marital_Together'] == 1:
        return 1
    else:
        return 0
data['In_relationship'] = data.apply(get_relationship, axis=1)
data.head()    


<a id="kmeans"></a>
# K-Means Clustering
K-means clustering is an unsupervised machine learning algorithm used to cluster data based on similarity. K-means clustering usually works well in practice and scales well to the large datasets.

In this section:
- Standardising data
- Principal Component Analysis (PCA)
- Elbow method
- Silhouette score analysis



In [ ]:
from sklearn.cluster import KMeans

## Standardising data
K-means clustering algorithm is based on the calculation of distances between data points to form clusters. When features have different scales, features with larger scales can disproportionately influence the distance calculation. There are various ways to standardise features, we will use <b> standard scaling </b>.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols_for_clustering = ['Income', 'MntTotal', 'In_relationship']
data_scaled = data.copy()
data_scaled[cols_for_clustering] = scaler.fit_transform(data[cols_for_clustering])
data_scaled[cols_for_clustering].describe()

The mean value for all colums is almost zero and the standard deviation is almost 1. All the data points were replaced by their z-scores.

## Principal Component Analysis (PCA)
PCA is a technique of dimensionality reduction. PCA takes the original features (dimensions) and create new features that capture the most variance of the data. 

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components = 2)
pca_res = pca.fit_transform(data_scaled[cols_for_clustering])
data_scaled['pc1'] = pca_res[:,0]
data_scaled['pc2'] = pca_res[:,1]

## Elbow method
The elbow method is a technique used to determine the optimal number of clusters (K) for K-means clustering algorithm.

In [ ]:
X = data_scaled[cols_for_clustering]
inertia_list = []
for K in range(2,10):
    inertia = KMeans(n_clusters=K, random_state=7).fit(X).inertia_
    inertia_list.append(inertia)

In [ ]:

plt.figure(figsize=[7,5])
plt.plot(range(2,10), inertia_list, color=(54 / 255, 113 / 255, 130 / 255))
plt.title("Inertia vs. Number of Clusters")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia")
plt.show()

Elbow method suggests 4 or 5 clusters. Let's check silhouette score.

## Silhouette score analysis
Silhouette score is a metric that used to assess the quality of clustering. A higher silhouette score indicates that the clusters are well-separated, while a lower score suggests that the clusters may overlap or are poorly defined.

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_list = []
for K in range(2,10):
    model = KMeans(n_clusters = K, random_state=7)
    clusters = model.fit_predict(X)
    s_avg = silhouette_score(X, clusters)
    silhouette_list.append(s_avg)

plt.figure(figsize=[7,5])
plt.plot(range(2,10), silhouette_list, color=(54 / 255, 113 / 255, 130 / 255))
plt.title("Silhouette Score vs. Number of Clusters")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Silhouette Score")
plt.show()

The highest silhouette score is for 4 clusters.

In [ ]:
model = KMeans(n_clusters=4, random_state = 7)
model.fit(data_scaled[cols_for_clustering])
data_scaled['Cluster'] = model.predict(data_scaled[cols_for_clustering])

<a id="exp_clust"></a>
# Exploration of Clusters
In this section:
- Visualisation of clusters
- Mean consumption of different product types by cluster
- Cluster sizes
- Income by cluster
- In_relationship feature by cluster



## Visualisation of clusters

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='pc1', y='pc2', data=data_scaled, hue='Cluster', palette='viridis')
plt.title('Clustered Data Visualization')
plt.xlabel('Principal Component 1 (pc1)')
plt.ylabel('Principal Component 2 (pc2)')
plt.legend(title='Clusters')

In [ ]:
data['Cluster'] = data_scaled.Cluster
data.groupby('Cluster')[cols_for_clustering].mean()

## Mean consumption of different product types by cluster

In [ ]:
mnt_data = data.groupby('Cluster')[cols_mnt].mean().reset_index()
mnt_data.head()

In [ ]:
melted_data = pd.melt(mnt_data, id_vars="Cluster", var_name="Product", value_name="Consumption")
plt.figure(figsize=(12, 6))
sns.barplot(x="Cluster", y="Consumption", hue="Product", data=melted_data, ci=None, palette="viridis")
plt.title("Product Consumption by Cluster")
plt.xlabel("Cluster")
plt.ylabel("Product Consumption")
plt.xticks(rotation=0)  
plt.legend(title="Product", loc="upper right")

plt.show()

## Cluster sizes

In [ ]:
cluster_sizes = data.groupby('Cluster')[['MntTotal']].count().reset_index()
plt.figure(figsize=(8,6))
sns.barplot(x='Cluster', y='MntTotal', data=cluster_sizes, palette = 'viridis')
plt.title('Cluster sizes')
plt.xlabel('Cluster')
plt.ylabel('MntTotal')


In [ ]:
total_rows = len(data)
cluster_sizes['Share%'] = round(cluster_sizes['MntTotal'] / total_rows*100,0)
cluster_sizes.head()


## Income by cluster
### Box plot

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Cluster', y='Income', data=data, palette='viridis')
plt.title('Income by cluster')
plt.xlabel('Cluster')
plt.ylabel('Income')
plt.legend(title='Clusters')

### Scatter plot

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Income', y='MntTotal', data=data, hue = 'Cluster', palette='viridis')
plt.title('Income by cluster')
plt.xlabel('Income')
plt.ylabel('MntTotal')
plt.legend(title='Clusters')

## In_relationship feature by cluster

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='Cluster', y='In_relationship', data=data, palette='viridis')
plt.title('In_relationship by cluster')
plt.xlabel('Cluster')
plt.ylabel('In_relationship')

<a id="results"></a>
# Results
This section contains the results of the K-means clustering analysis, which aimed to identify distinct customer segments based on the total amount of purchases they made (MntTotal). The analysis utilised 'Income' and 'In_relationship' features.

## Optimal number of clusters = 4
The Elbow Method and Silhouette Analysis suggested 4 clusters (k=4). The elbow method highlighted the number of 4 or 5 clusters as a reasonable number of clusters. The silhouette score analysis revealed a peak silhouette score for k=4.

## Cluster Characteristics
### Cluster 0: High value customers in relationship (either married or together)
- This cluster represents 26% of the customer base
- These customers have high income and they are in a relationship

### Cluster 1: Low value single customers
- This cluster represents 21% of the customer base
- These customers have low income and they are single

### Cluster 2: High value single customers
- This cluster represents 15% of the customer base
- These customers have high income and they are single

### Cluster 3: Low value customers in relationship
- This cluster represents 39% of the customer base
- These customers have low income and they are in a relationship




<a id="recommendations"></a>
# Recommendations
Based on the clusters, tailored marketing strategies can be created. Customers from these segments will have different interests and product preferences.

## Marketing Strategies for Each Cluster

### Cluster 0: High value customers in relationship (either married or together)
- Preliminary analysis showed that high income customers buy more wines and fruits. 
- A tailored campaign to promote high quality wines may bring good results. 
- This cluster contains customers in relationship, family-oriented promo-images should be quite effective for this audience.

### Cluster 1: Low value single customers
- Promos with discounts and coupons may bring good results for this targeted group.
- Loyalty program may stimulate these customers to purchase more often. 

### Cluster 2: High value single customers
- Similar to the Cluster 0, these customers buy a lot of wines and fruits.
- This cluster contains single customers. Promo images with friends, parties or single trips may be more efficient for single customers

### Cluster 3: Low value customers in relationship
- This cluster has the highest percentage of our customers (39%).
- Family offers and discounts may influence these customers to make more purchases

## Opportunities for the further analysis
- Further exploration on how children influence on the consumed products 
- Further analysis on the influence of education
- analysis of frequent buyers
- Analysis of sales channels, e.g. store, website, etc.
- Analysis of the response to the marketing campaigns
- It would be great to add gender data to the dataset
- Test different clustering algorithms


# Thank you!